<h1>Data Load</h1>

In [521]:
#Import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import urllib
import re

import torch
import keras
from keras.applications.vgg16 import VGG16

import warnings
warnings.filterwarnings('ignore')

In [104]:
#data read
df = pd.read_csv('mergeddata.csv')
df

,asin,image,category,description,title
0,0511189877,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The CLIKR-5 UR5U-8780L remote control is des...,CLIKR-5 Time Warner Cable Remote Control UR5U-...
1,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi..."
2,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi..."
3,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi..."
4,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi..."
...,...,...,...,...,...
112061,B01HICMC3A,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Ne...","[""Type-C to 4-Port USB 3.0 Hub 4-port USB 3.0 ...","USB-C Hub, EC Technology 4 ports USB 3.0 hub f..."
112062,B01HICMC3A,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Ne...","[""Type-C to 4-Port USB 3.0 Hub 4-port USB 3.0 ...","USB-C Hub, EC Technology 4 ports USB 3.0 hub f..."
112063,B01HIS365W,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Unlock the potential of your iOS devices wit...,Lightning microSD Card Reader for iPhone and iPad
112064,B01HIW2XPM,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...","[""This enclosure frames the Pine64 while leavi...",Zebra Case for Pine64 ~ Black Ice Slim C4Labs


In [105]:
#create function to obtain image data and convert into generator
def get_image_data(url_list):
    url_list = url_list.replace('\'','').replace('[','').replace(']','').split(',')
    for url in url_list:
        yield plt.imread(urllib.request.urlopen(url), format = 'jpeg')

In [106]:
df['loaded_image'] = df['image'].apply(get_image_data)
df.head()

,asin,image,category,description,title,loaded_image
0,0511189877,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The CLIKR-5 UR5U-8780L remote control is des...,CLIKR-5 Time Warner Cable Remote Control UR5U-...,<generator object get_image_data at 0x000001FD...
1,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...
2,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...
3,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...
4,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...


In [222]:
df['loaded_image'][0]

<generator object get_image_data at 0x000001FD07766E48>

In [107]:
#Duplicates are identified. Drop them
df = df.drop_duplicates('asin')
df.head()

,asin,image,category,description,title,loaded_image
0,0511189877,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The CLIKR-5 UR5U-8780L remote control is des...,CLIKR-5 Time Warner Cable Remote Control UR5U-...,<generator object get_image_data at 0x000001FD...
1,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...
8,106171327X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Sandisk SDSDQUA-064G-A11 Professional Ultra ...,Sandisk SDSDQUA-064G-A11 Professional Ultra 64...,<generator object get_image_data at 0x000001FD...
10,140053271X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'eBook Readers & Accessories',...","[""The NOOK Simple Touch eReader allows you to ...",Barnes & Noble Nook Simple Touch eBook Reader ...,<generator object get_image_data at 0x000001FD...
11,1400501466,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...","[""With the ENHANCED Duo-boot Nook Color you ge...","Barnes & Noble NOOK Tablet 16gb (Color, BNTV250)",<generator object get_image_data at 0x000001FD...


In [108]:
#there are only around 39965 data in total
len(df)

39965

In [109]:
#print example
for i in df.iloc[0]:
    print(i)
    print('\n')

0511189877


['https://images-na.ssl-images-amazon.com/images/I/81XYkHdfrjL._SY88.jpg']


['Electronics', 'Accessories & Supplies', 'Audio & Video Accessories', 'Remote Controls & Accessories', 'Remote Controls']


['The CLIKR-5 UR5U-8780L remote control is designed to operate the CISCO/SA, Pioneer, Pace Micro, Samsung and Motorola digital set tops.  Plus the majority of TV, VCR, DVD and AUX equipment on the market as follows: Cable Set-top Boxes, VCRs and TV/VCRs, Televisions, DVD Players, Audio Equipment, Media PC, etc.\n\n\nContact Information\nfor MSO\'s\nLet\'s discuss your needs in detail.\n \nTime Warner Cable CLIKR-5 \n\nInteractive Tutorial - English and Spanish \n\nProgramming Instructions and Codes - English \n\nThe Time Warner Cable CLIKR-5 is designed to offer complete control for all digital, HD and DVR set-top boxes. The UR5U-8780L is factory programmed to operate Scientific Atlanta/Cisco and Samsung set-top boxes while the UR5U-8790L is factory programmed for Motorola s

<h1>Data Cleaning</h1>

In [110]:
df.isnull().sum()

asin               0
image              0
category           0
description     6321
title              0
loaded_image       0
dtype: int64

In [111]:
len(df)

39965

In [150]:
#check for how many unique elements in the category
unique_category = set()
cat_count = []

for i in df['category']:
    for j in i.strip('[]').split(','):
        j = j.strip('\'').strip(' \'')
        unique_category.add(j)
        cat_count.append(j)

cat_count = pd.Series(cat_count)

unique_category

{'Shoulder Bags',
 '',
 'unisex vintage look',
 'Two-Way Radios',
 'Security & Surveillance',
 'Diffusers',
 'PERFECT GIFT IDEA: Give this bag as a gift for someone with a new job',
 'SCSI Adapters',
 'Included libraries and example code for both SD and RTC',
 'as it gives a Classic impression to your colleagues and your friends buy this unisex bag',
 'easily dividing the camera from lens and flash.',
 '1* zipper pocket on the front',
 'Size :44H x 30W x 16Dcm(11.8" x 6.3" x 17.3")',
 'Photo Studio',
 'MP3 & MP4 Players',
 'H:45cm/17.8"',
 'Thunderbolt Cables',
 'please feel free to contact us if any problem.',
 'Super convenient',
 'Earbud Headphones',
 'Combines rugged elements from the Defender Series with the thin silicone skin from the Impact Series',
 'Line Conditioners',
 'anti reflective coatings and patented focusing power 0.20 mag.',
 'Video Converters',
 'alleviating body strain',
 'Mirrorless Camera Lenses',
 'Size:106cm*68cm/41.7"*26.7"',
 'Video Transmission Systems',
 'C

In [146]:
len(unique_category)

2175

In [151]:
#top counts for tags
cat_count.value_counts()

Electronics                                                                                      39965
Computers & Accessories                                                                          16158
Camera & Photo                                                                                    9136
Accessories & Supplies                                                                            4889
Computer Accessories & Peripherals                                                                4484
                                                                                                 ...  
Quality Zipper. Panel-loading style zipper rucksack. Double-headed zippers for easiest access        1
Padded compartment protects iPad or tablet up to 11                                                  1
Man-Made Materials                                                                                   1
accurate typing.Magnets activate sleep/wake function to preserve battery 

In [157]:
used_cat = cat_count.value_counts()[:200].index
used_cat

Index(['Electronics', 'Computers & Accessories', 'Camera & Photo',
       'Accessories & Supplies', 'Computer Accessories & Peripherals',
       'Audio & Video Accessories', 'Bags', 'Cases & Sleeves',
       'Tablet Accessories', 'Accessories',
       ...
       'Component Subwoofers', 'Hard Drive Bags & Cases',
       'Compact Radios & Stereos', 'Vehicle Mounts', 'Cleaning & Repair',
       'Camera', 'Water Cooling Systems', 'Selector Switch Boxes',
       'Binoculars', 'Projection Screens'],
      dtype='object', length=200)

In [180]:
cat_count.value_counts()[:200].to_csv('top_200_cat.csv')

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  """Entry point for launching an IPython kernel.


In [487]:
cat_clean = []

for ix_i, i in enumerate(df['category']):
    if ix_i % 1000 == 0:
        print('Row number: ', ix_i)
    
    i = i.strip('[]').replace('\'','').split(',')
    temp = []
    for j in i:
        j = j.strip(' ')
        temp.append(j)
    cat_clean.append(temp)
#     print(i)

#category_clean is in form of list
df['category_clean'] = cat_clean

Row number:  0
Row number:  1000
Row number:  2000
Row number:  3000
Row number:  4000
Row number:  5000
Row number:  6000
Row number:  7000
Row number:  8000
Row number:  9000
Row number:  10000
Row number:  11000
Row number:  12000
Row number:  13000
Row number:  14000
Row number:  15000
Row number:  16000
Row number:  17000
Row number:  18000
Row number:  19000
Row number:  20000
Row number:  21000
Row number:  22000
Row number:  23000
Row number:  24000
Row number:  25000
Row number:  26000
Row number:  27000
Row number:  28000
Row number:  29000
Row number:  30000
Row number:  31000
Row number:  32000
Row number:  33000
Row number:  34000
Row number:  35000
Row number:  36000
Row number:  37000
Row number:  38000
Row number:  39000


In [468]:
len(unique_category)

2175

In [497]:
#need to run this cell several times until no more printout on j
cat_clean_sorted = []

for ix_i, i in enumerate(df['category_clean']):
    if ix_i % 1000 == 0:
        print('Row number: ', ix_i)
        
    for j in i:
        if j in used_cat:
            pass
        else:
            print(j)
            i.remove(j)
    cat_clean_sorted.append(i)
            
df['category_clean_sorted'] = cat_clean_sorted

Row number:  0
Row number:  1000
Row number:  2000
Row number:  3000
Row number:  4000
Row number:  5000
Row number:  6000
Row number:  7000
Row number:  8000
Row number:  9000
Row number:  10000
Row number:  11000
Row number:  12000
Row number:  13000
Row number:  14000
Row number:  15000
Row number:  16000
Row number:  17000
Row number:  18000
Row number:  19000
Row number:  20000
Row number:  21000
Row number:  22000
Row number:  23000
Row number:  24000
Row number:  25000
Row number:  26000
Row number:  27000
Row number:  28000
Row number:  29000
Row number:  30000
Row number:  31000
Row number:  32000
Row number:  33000
Row number:  34000
Row number:  35000
Row number:  36000
Row number:  37000
Row number:  38000
Row number:  39000


In [498]:
#check for how many unique elements in the category
unique_category = set()
cat_count = []

for i in df['category_clean_sorted']:
    for j in i:
#         j = j.strip('\'').strip(' \'')
        unique_category.add(j)
        cat_count.append(j)

cat_count = pd.Series(cat_count)

unique_category

{'AC Adapters',
 'Accessories',
 'Accessories & Supplies',
 'Accessory Kits',
 'Adapters',
 'Amplifiers',
 'Antennas',
 'Audio & Video Accessories',
 'Audio Cables',
 'Backgrounds',
 'Backpacks',
 'Bags',
 'Bags & Cases',
 'Batteries',
 'Batteries & Chargers',
 'Battery Chargers',
 'Binocular',
 'Binoculars',
 'Binoculars & Scopes',
 'Bluetooth & FM Transmitters',
 'Bullet Cameras',
 'CB & Two-Way Radios',
 'Cables & Interconnects',
 'Camcorders',
 'Camera',
 'Camera & Camcorder Straps',
 'Camera & Photo',
 'Camera Batteries',
 'Camera Cases',
 'Camera Lenses',
 'Camera Mounts & Clamps',
 'Car & Vehicle Electronics',
 'Car Audio',
 'Car Electronics',
 'Car Safety & Security',
 'Car Stereo Receivers',
 'Car Video',
 'Case Fans',
 'Cases',
 'Cases & Sleeves',
 'Cat 6 Cables',
 'Chargers & Accessories',
 'Chargers & Adapters',
 'Cleaning & Repair',
 'Compact Radios & Stereos',
 'Complete Tripods',
 'Component Subwoofers',
 'Computer & Monitor Mounts',
 'Computer Accessories & Peripherals'

In [499]:
len(unique_category)

200

In [500]:
for ix_i, i in enumerate(df['category_clean_sorted']):
    if ix_i % 1000 == 0:
        print('Row number: ', ix_i)
        
    for ix_j, j in enumerate(i):
        if j not in df.columns:
            df[j] = 0
            df[j][ix_i] = 1
        elif j in df.columns:
            df[j][ix_i] = 1

df

Row number:  0
Row number:  1000
Row number:  2000
Row number:  3000
Row number:  4000
Row number:  5000
Row number:  6000
Row number:  7000
Row number:  8000
Row number:  9000
Row number:  10000
Row number:  11000
Row number:  12000
Row number:  13000
Row number:  14000
Row number:  15000
Row number:  16000
Row number:  17000
Row number:  18000
Row number:  19000
Row number:  20000
Row number:  21000
Row number:  22000
Row number:  23000
Row number:  24000
Row number:  25000
Row number:  26000
Row number:  27000
Row number:  28000
Row number:  29000
Row number:  30000
Row number:  31000
Row number:  32000
Row number:  33000
Row number:  34000
Row number:  35000
Row number:  36000
Row number:  37000
Row number:  38000
Row number:  39000


,asin,image,category,description,title,loaded_image,category_clean,category_clean_sorted,Electronics,Accessories & Supplies,...,Internal Solid State Drives,Screen Protectors,Sports & Action Video Cameras,Water Cooling Systems,Skins & Decals,Keyboard Cases,Wearable Technology,Smartwatches,Lightning Cables,Single Board Computers
0,0511189877,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The CLIKR-5 UR5U-8780L remote control is des...,CLIKR-5 Time Warner Cable Remote Control UR5U-...,<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio & ...","[Electronics, Accessories & Supplies, Audio & ...",1,1,...,0,0,0,0,0,0,0,0,0,0
1,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio & ...","[Electronics, Accessories & Supplies, Audio & ...",1,1,...,0,0,0,0,0,0,0,0,0,0
8,106171327X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Sandisk SDSDQUA-064G-A11 Professional Ultra ...,Sandisk SDSDQUA-064G-A11 Professional Ultra 64...,<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compute...","[Electronics, Computers & Accessories, Compute...",1,0,...,0,0,0,0,0,0,0,0,0,0
10,140053271X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'eBook Readers & Accessories',...","[""The NOOK Simple Touch eReader allows you to ...",Barnes & Noble Nook Simple Touch eBook Reader ...,<generator object get_image_data at 0x000001FD...,"[Electronics, eBook Readers & Accessories]","[Electronics, eBook Readers & Accessories]",0,0,...,0,0,0,0,0,0,0,0,0,0
11,1400501466,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...","[""With the ENHANCED Duo-boot Nook Color you ge...","Barnes & Noble NOOK Tablet 16gb (Color, BNTV250)",<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compute...","[Electronics, Computers & Accessories, Compute...",0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112060,B01HI9ZK8M,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['Products Feature: The MINI HDMI to AV signa...,Mifine 1080P MINI HDMI to AV 3RCA CVBs Composi...,<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio & ...","[Electronics, Accessories & Supplies, Audio & ...",0,0,...,0,0,0,0,0,0,0,0,0,0
112061,B01HICMC3A,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Ne...","[""Type-C to 4-Port USB 3.0 Hub 4-port USB 3.0 ...","USB-C Hub, EC Technology 4 ports USB 3.0 hub f...",<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Network...","[Electronics, Computers & Accessories, Network...",0,0,...,0,0,0,0,0,0,0,0,0,0
112063,B01HIS365W,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Unlock the potential of your iOS devices wit...,Lightning microSD Card Reader for iPhone and iPad,<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compute...","[Electronics, Computers & Accessories, Compute...",0,0,...,0,0,0,0,0,0,0,0,0,0
112064,B01HIW2XPM,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...","[""This enclosure frames the Pine64 while leavi...",Zebra Case for Pine64 ~ Black Ice Slim C4Labs,<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compute...","[Electronics, Computers & Accessories, Compute...",0,0,...,0,0,0,0,0

In [501]:
#save df
# df.to_csv('final_table.csv')

In [518]:
df['Electronics'].max()

1

In [520]:
#sanity check on the OHE columns
colsmax=[]

for i in df:
    if df[i].dtype == 'int64':
        colsmax.append(df[i].max())
    else:
        pass
    
max(colsmax)

1

<h1>Modelling</h1>

In [ ]:
loaded_img = df['image'].apply(get_image_data)
print_img = iter(list(loaded_img))

img_size = []

while True:
    img_size.append(list(next(print_img))[0].shape)

In [257]:
df['category_clean'][0]

['Electronics', ' Remote Controls']

In [247]:
cat_clean_sorted

[['Electronics', ' Remote Controls'],
 ['Electronics', ' TV Ceiling & Wall Mounts'],
 ['Electronics', ' Micro SD Cards'],
 ['Electronics'],
 ['Electronics'],
 ['Electronics'],
 ['Electronics'],
 ['Electronics'],
 ['Electronics', ' Cases & Sleeves'],
 ['Electronics']]

In [250]:
df['category'][0]

"['Electronics', 'Accessories & Supplies', 'Audio & Video Accessories', 'Remote Controls & Accessories', 'Remote Controls']"

In [234]:
a = ['aa', 'bb','cc']
b = ['aa','bb', 'cc', 'dd', 'ee']
li = []

for i in b:
    print(i)
    if i in a:
        print(i, 'yes')
        pass
    elif i not in a:
        print(i, 'no')
        b.remove(i)
        pass
        
b

aa
aa yes
bb
bb yes
cc
cc yes
dd
dd no


['aa', 'bb', 'cc', 'ee']

In [218]:
#no empty category
for i in df['category_clean_sorted']:
    if len(i) == 0:
        print(i)

In [221]:
for ix_i, i in enumerate(df['category_clean_sorted']):
    for ix_j, j in enumerate(i):
        if j not in df.columns:
            df[j] = 0
            df[j][ix_i] = 1
        elif j in df.columns:
            df[j][ix_i] = 1

print(len(df))
df

39965


,asin,image,category,description,title,loaded_image,category_clean,category_clean_sorted,Electronics,Accessories & Supplies,...,Guaranteed to protect your Apple IPad Air 2,and to last for a long time.,Outside Measurements (In inches): 10 (Length) x 13 (Height) x 4.5 (Deep),Inside Measurements (In inches): 10 (Length) x 13 (Height) x 4.5 (Deep),Weight: 1.0 kgs/ 2.5 Lbs (approx),Front: 2 small pockets and 1 zipper pocket Side: 2 small pockets Inside: 1 zipper pocket,"3 padded compartments 1.5"" wide and 70"" long adjustable leather shoulder strap with hand carry","""Material Used- Genuine Goat Leather Processed",no bad smell Naturally tanned with sunflower oil only,"no use of dye or chemicals Genuine Nickle Polished Solid Steel Accessories Inside is lined with sturdy but soft canvas"""
0,0511189877,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The CLIKR-5 UR5U-8780L remote control is des...,CLIKR-5 Time Warner Cable Remote Control UR5U-...,<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio ...","[Electronics, Accessories & Supplies, Audio ...",1,1,...,0,0,0,0,0,0,0,0,0,0
1,0972683275,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['The videosecu TV mount is a mounting solutio...,"VideoSecu 24"" Long Arm TV Wall Mount Low Profi...",<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio ...","[Electronics, Accessories & Supplies, Audio ...",1,1,...,0,0,0,0,0,0,0,0,0,0
8,106171327X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Sandisk SDSDQUA-064G-A11 Professional Ultra ...,Sandisk SDSDQUA-064G-A11 Professional Ultra 64...,<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compu...","[Electronics, Computers & Accessories, Compu...",1,0,...,0,0,0,0,0,0,0,0,0,0
10,140053271X,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'eBook Readers & Accessories',...","[""The NOOK Simple Touch eReader allows you to ...",Barnes & Noble Nook Simple Touch eBook Reader ...,<generator object get_image_data at 0x000001FD...,"[Electronics, eBook Readers & Accessories, e...","[Electronics, eBook Readers & Accessories, e...",0,0,...,0,0,0,0,0,0,0,0,0,0
11,1400501466,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...","[""With the ENHANCED Duo-boot Nook Color you ge...","Barnes & Noble NOOK Tablet 16gb (Color, BNTV250)",<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compu...","[Electronics, Computers & Accessories, Compu...",0,1,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
112060,B01HI9ZK8M,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Accessories & Supplies', 'Aud...",['Products Feature: The MINI HDMI to AV signa...,Mifine 1080P MINI HDMI to AV 3RCA CVBs Composi...,<generator object get_image_data at 0x000001FD...,"[Electronics, Accessories & Supplies, Audio ...","[Electronics, Accessories & Supplies, Audio ...",0,0,...,0,0,0,0,0,0,0,0,0,0
112061,B01HICMC3A,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Ne...","[""Type-C to 4-Port USB 3.0 Hub 4-port USB 3.0 ...","USB-C Hub, EC Technology 4 ports USB 3.0 hub f...",<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Netwo...","[Electronics, Computers & Accessories, Netwo...",0,0,...,0,0,0,0,0,0,0,0,0,0
112063,B01HIS365W,['https://images-na.ssl-images-amazon.com/imag...,"['Electronics', 'Computers & Accessories', 'Co...",['Unlock the potential of your iOS devices wit...,Lightning microSD Card Reader for iPhone and iPad,<generator object get_image_data at 0x000001FD...,"[Electronics, Computers & Accessories, Compu...","[Electronics, Compu

In [ ]:
df['category_clean_sorted']

In [217]:
a = ['aa', 'bb']
b = pd.DataFrame({'c':[1,2], 'd':['dd','ee']})

for ix, i in enumerate(a):
    if i not in b.columns:
        b[i] = 0
        b[i][ix] = 1
#         b.iloc[ix][i] = 1
    elif i in b.columns:
        b[i][ix] = 1

b

D:\ProgramFiles\Anaconda3\lib\site-packages\ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys


,c,d,aa,bb
0,1,dd,1,0
1,2,ee,0,1


In [212]:
b.iloc[0][0]

1

In [189]:
for i in df['category_clean'][0]:
    i = i.strip(' ')
    if i in used_cat:
        print(i, 'yes')
    elif i not in used_cat:
        print(i, 'no')

Electronics yes
Accessories & Supplies yes
Audio & Video Accessories yes
Remote Controls & Accessories yes
Remote Controls yes


In [183]:
'a' in 'abc'

True